<a href="https://colab.research.google.com/github/zoefleischer/data-prework/blob/master/week2_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 2 Project
You were hired by Ironhack to perform an Analytics Consulting Project entitled: competitive landscape.

Your mission is to create and populate an appropriate database with many coding schools that are our competition, as well as design an suitable queries that answer business questions of interest (to be defined by you)


**Suggested Steps in the Project:**


*   Read this notebook and understand each function. Comment the code appropriately

*   Populate the list of schools with a wider variety of schools (how are you going to get the school ID?)

* Take a look at the obtained dataframes. What dimensions do you have? what keys do you have? how could the different dataframes be connected?

* Go back to the drawing board and try to create an entity relationship diagram for tables available

* Once you have the schemas you want, you will need to:
  - create the suitable SQL queries to create the tables and populate them
  - run these queries using the appropriate Python connectors
  
* Bonus: How will this datamodel be updated in the future? Please write auxiliary functions that test the database for data quality issues. For example: how could you make sure you only include the most recent comments when you re-run the script?


# Suggested Deliverables

* 5-6 minute presentation of data model created, decision process and business analysis proposed

* exported .sql file with the final schema

* Supporting python files used to generate all logic

* High level documentation explaining tables designed and focusing on update methods

In [ ]:
# you must populate this dict with the schools required -> try talking to the teaching team about this


schools = {   
'ironhack' : 10828,
'app-academy' : 10525,
'springboard' : 11035    
}

import re
import pandas as pd
from pandas.io.json import json_normalize
import requests



def get_comments_school(school):
  TAG_RE = re.compile(r'<[^>]+>')
  # defines url to make api call to data -> dynamic with school if you want to scrape competition
  url = "https://www.switchup.org/chimera/v1/school-review-list?mainTemplate=school-review-list&path=%2Fbootcamps%2F" + school + "&isDataTarget=false&page=3&perPage=10000&simpleHtml=true&truncationLength=250"
  #makes get request and converts answer to json
  data = requests.get(url).json()
  #converts json to dataframe
  reviews =  pd.DataFrame(data['content']['reviews'])
  
  #aux function to apply regex and remove tags
  def remove_tags(x):
    return TAG_RE.sub('',x)
  reviews['review_body'] = reviews['body'].apply(remove_tags)
  reviews['school'] = school
  return reviews



In [ ]:
# could you write this as a list comprehension? ;)
comments = []

for school in schools.keys():
    print(school)
    comments.append(get_comments_school(school))

comments = pd.concat(comments)

ironhack
app-academy
springboard


In [ ]:
comments

,id,name,anonymous,hostProgramName,graduatingYear,isAlumni,jobTitle,tagline,body,createdAt,queryDate,program,user,overallScore,comments,overall,curriculum,jobSupport,review_body,school
0,266037,Anonymous,True,None,2020.0,True,,Great experience as expected :),"<span class=""truncatable""><p>Excellent choice ...",7/3/2020,2020-07-03,Data Analytics Bootcamp,{'image': None},5.0,[],5.0,5.0,5.0,Excellent choice to get introduced into Data A...,ironhack
1,266009,Manon Philippe,False,None,2020.0,False,Designer,A very complete bootcamp !,"<span class=""truncatable""><p>I was unconvinced...",7/2/2020,2020-07-02,UX/UI Design Bootcamp,{'image': None},4.7,[],5.0,5.0,4.0,I was unconvinced when I joined this formation...,ironhack
2,265960,Tim Stephens,False,None,2020.0,False,full stack web developer/teacher,Hugely rewarding experience that gave me a job...,"<span class=""truncatable""><p>Pros: <br>- Learn...",7/1/2020,2020-07-01,Web Development Bootcamp,{'image': None},4.7,[],5.0,4.0,5.0,Pros: - Learning javascript to the level of a ...,ironhack
3,265948,Anonymous,True,None,2020.0,False,,"Me gustó, aprendí muchas cosas nuevas","<span class=""truncatable""><p>Mi experiencia en...",6/30/2020,2020-06-30,UX/UI Design Part-Time,{'image': None},4.0,[],4.0,4.0,4.0,"Mi experiencia en Ironhack fue muy buena, apre...",ironhack
4,265947,Alberto Ibarra,False,None,2020.0,False,Data Analyst,The best investment of time and money,"<span class=""truncatable""><p>I wanted to make ...",6/30/2020,2020-06-30,Data Analytics Bootcamp,{'image': None},5.0,[],5.0,5.0,5.0,"I wanted to make a career change for a while, ...",ironhack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,234894,Stephanie S.,False,None,2015.0,True,Website Manager-UX Lead,Excellent and comprehensive UX course,"<span class=""truncatable""><p>The UX Design Wor...",12/15/2016,2016-12-15,UX Design,{'image': None},5.0,[],5.0,5.0,5.0,The UX Design Workshop gave me a great underst...,springboard
592,234877,Joe Fang,False,None,2015.0,True,UX Designer,Springboard helped me start my career in UX,"<span class=""truncatable""><p>Just some backgro...",12/15/2016,2016-12-15,UX Design,{'image': None},5.0,[],5.0,5.0,5.0,"Just some background information on myself, I ...",springboard
593,234838,Zeina,False,None,2016.0,True,Senior Graphic Designer,Highly recommended,"<span class=""truncatable""><p>I enjoyed the pro...",12/15/2016,2016-12-15,UX Design,{'image': None},3.7,[],4.0,4.0,3.0,"I enjoyed the program very much, gained a lot ...",springboard
594,242681,Jean,False,None,2016.0,True,Lead Designer/Creative Director,A way to stay current for the highly motivated!,"<span class=""truncatable""><p>This was my first...",12/12/2016,2016-12-12,UX Design,{'image': None},5.0,[],5.0,5.0,5.0,This was my first online experience and it did...,springboard


In [ ]:
from pandas.io.json import json_normalize

def get_school_info(school, school_id):
    url = 'https://www.switchup.org/chimera/v1/bootcamp-data?mainTemplate=bootcamp-data%2Fdescription&path=%2Fbootcamps%2F'+ str(school) + '&isDataTarget=false&bootcampId='+ str(school_id) + '&logoTag=logo&truncationLength=250&readMoreOmission=...&readMoreText=Read%20More&readLessText=Read%20Less'

    data = requests.get(url).json()

    data.keys()

    courses = data['content']['courses']
    courses_df = pd.DataFrame(courses, columns= ['courses'])

    locations = data['content']['locations']
    locations_df = json_normalize(locations)

    badges_df = pd.DataFrame(data['content']['meritBadges'])
    
    website = data['content']['webaddr']
    description = data['content']['description']
    logoUrl = data['content']['logoUrl']
    school_df = pd.DataFrame([website,description,logoUrl]).T
    school_df.columns =  ['website','description','LogoUrl']

    locations_df['school'] = school
    courses_df['school'] = school
    badges_df['school'] = school
    school_df['school'] = school
    
    # how could you write a similar block of code to the above in order to record the school ID?

    return locations_df, courses_df, badges_df, school_df

locations_list = []
courses_list = []
badges_list = []
schools_list = []

for school, id in schools.items():
    print(school)
    a,b,c,d = get_school_info(school,id)
    locations_list.append(a)
    courses_list.append(b)
    badges_list.append(c)
    schools_list.append(d)



ironhack


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


app-academy
springboard


In [ ]:
locations = pd.concat(locations_list)
locations.head()

,id,description,country.id,country.name,country.abbrev,city.id,city.name,city.keyword,state.id,state.name,state.abbrev,state.keyword,school
0,15901,"Berlin, Germany",57.0,Germany,DE,31156.0,Berlin,berlin,NaN,NaN,NaN,NaN,ironhack
1,16022,"Mexico City, Mexico",29.0,Mexico,MX,31175.0,Mexico City,mexico-city,NaN,NaN,NaN,NaN,ironhack
2,16086,"Amsterdam, Netherlands",59.0,Netherlands,NL,31168.0,Amsterdam,amsterdam,NaN,NaN,NaN,NaN,ironhack
3,16088,"Sao Paulo, Brazil",42.0,Brazil,BR,31121.0,Sao Paulo,sao-paulo,NaN,NaN,NaN,NaN,ironhack
4,16109,"Paris, France",38.0,France,FR,31136.0,Paris,paris,NaN,NaN,NaN,NaN,ironhack


In [ ]:
courses = pd.concat(courses_list)
courses.head(10)

,courses,school
0,Data Analytics Bootcamp,ironhack
1,Data Analytics Part-Time,ironhack
2,UX/UI Design Bootcamp,ironhack
3,UX/UI Design Part-Time,ironhack
4,Web Development Bootcamp,ironhack
5,Web Development Part-Time,ironhack
0,Bootcamp Prep,app-academy
1,Software Engineer Track: In-Person,app-academy
2,Software Engineer Track: Online,app-academy
0,Data Analytics Career Track,springboard


In [ ]:
badges = pd.concat(badges_list)
badges.head()

,name,keyword,description,school
0,Available Online,available_online,<p>School offers fully online courses</p>,ironhack
1,Verified Outcomes,verified_outcomes,<p>School publishes a third-party verified out...,ironhack
2,Flexible Classes,flexible_classes,<p>School offers part-time and evening classes...,ironhack
0,Available Online,available_online,<p>School offers fully online courses</p>,app-academy
1,Job Guarantee,job_guarantee,<p>School guarantees job placement</p>,app-academy


In [ ]:
# any data cleaning still missing here? take a look at the description
schools = pd.concat(schools_list)
schools.head()

,website,description,LogoUrl,school
0,www.ironhack.com/en,"<span class=""truncatable""><p>Ironhack is a glo...",https://d92mrp7hetgfk.cloudfront.net/images/si...,ironhack
0,appacademy.io,"<span class=""truncatable""><p>App Academy is a ...",https://d92mrp7hetgfk.cloudfront.net/images/si...,app-academy
0,www.springboard.com/?utm_source=switchup&utm_m...,"<span class=""truncatable""><p>Springboard is an...",https://d92mrp7hetgfk.cloudfront.net/images/si...,springboard
